In [1]:
import pandas as pd
import numpy as np
import re

# 경고 메세지 없애기
import warnings
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt
%matplotlib inline

import matplotlib.font_manager as fm
path = 'C:\Windows\Fonts\malgunbd.ttf'
font_name = fm.FontProperties(fname=path).get_name()
print(font_name)
plt.rc('font', family=font_name)

import seaborn as sns

# 지도 표현
import folium

# ggplot
from plotnine import *

Malgun Gothic


In [2]:
shop_2018 = pd.read_csv('data/소상공인시장진흥공단_상가업소정보_201806_03.csv', encoding='cp949')
shop_2018.head()

,상가업소번호,상호명,지점명,상권업종대분류코드,상권업종대분류명,상권업종중분류코드,상권업종중분류명,상권업종소분류코드,상권업종소분류명,표준산업분류코드,...,건물관리번호,건물명,도로명주소,구우편번호,신우편번호,동정보,층정보,호정보,경도,위도
0,28482144,부뚜막삼겹살,NaN,Q,음식,Q01,한식,Q01A02,갈비/삼겹살,I56111,...,4173025022100420000057910,NaN,경기도 여주시 여흥로109번길 10,469102,12622.0,NaN,1,NaN,127.635942,37.296409
1,28497846,나이키영통점,영통점,D,소매,D05,의복의류,D05A02,캐쥬얼/스포츠의류,G47416,...,4111710500109960004001202,NaN,경기도 수원시 영통구 봉영로 1612,443470,16704.0,NaN,7,NaN,127.074963,37.254463
2,28499066,눈빛안경.콘택트,NaN,D,소매,D18,사진/광학/정밀기기소매,D18A04,안경원,G47822,...,4136031021101050001000001,NaN,경기도 남양주시 별내면 청학로응달길 51-12,472811,12077.0,NaN,1,NaN,127.129433,37.709625
3,28493571,설국장,NaN,O,숙박,O02,모텔/여관/여인숙,O02A01,모텔/여관/여인숙,I55112,...,4146510100109410000025298,NaN,경기도 용인시 수지구 풍덕천로190번길 19-1,448170,16833.0,NaN,1,NaN,127.102445,37.324824
4,28518376,파고망화장품,NaN,D,소매,D16,화장품소매,D16A01,화장품판매점,G47813,...,4159011600008240007020102,NaN,경기도 화성시 떡전골로 103,445390,18390.0,NaN,1,NaN,127.033253,37.207750


In [3]:
coffee = shop_2018[shop_2018['상권업종소분류명'].str.contains('커피')]
coffee.shape

(23845, 39)

In [4]:
coffee['시도명'].value_counts()

경기도     18213
강원도      3339
충청북도     2293
Name: 시도명, dtype: int64

In [5]:
df_gg = coffee.loc[coffee['시도명'].str.startswith('경기')].copy()
df_gg.shape

(18213, 39)

In [6]:
df_gg['상호명'].value_counts()

이디야커피              375
간단하지만특별한화피디Cafe    266
커피에반하다             236
스타벅스               190
요거프레소              156
투썸플레이스             131
빽다방                114
커피베이               108
쥬씨                  95
커피마마                74
카페베네                67
엔제리너스               65
할리스커피               64
토프레소                61
공차                  50
셀렉토커피               48
설빙                  47
더카페                 42
커피하우스               40
탐앤탐스                38
파스쿠찌                37
커피빈                 36
커피나무                35
커피콩                 30
메가커피                29
더착한커피               28
청년다방                27
떼루와                 27
카페루앤비               20
카페인                 20
                  ... 
프레소커피                1
신다방                  1
마리나커피하우스             1
엔젤리안                 1
데이스타터커피              1
바누리커피앤로스터            1
로드카페                 1
바다와풍경                1
한향림갤러리               1
로렐커피앤플레이스            1
커피베이조원점              1
베럴커피                 1
작은모퉁이      

In [7]:
df_cafe = df_gg[df_gg['상호명'].str.contains('스타벅스|starbucks|STARBUCKS|이디야|ediya|EDIYA')]
df_cafe.shape

(725, 39)

In [8]:
df_cafe['브랜드명'] = ''

In [9]:
df_cafe.loc[df_cafe['상호명'].str.contains('이디야|ediya|EDIYA'), '브랜드명'] = '이디야'
df_cafe.loc[df_cafe['상호명'].str.contains('스타벅스|starbucks|STARBUCKS'), '브랜드명'] = '스타벅스'
df_cafe[['상호명','브랜드명' ]].head()

,상호명,브랜드명
3115,스타벅스,스타벅스
3565,이디야커피,이디야
14031,이디야커피숍,이디야
19110,스타벅스,스타벅스
19296,이디야커피가현초교점,이디야


In [10]:
df_cafe['브랜드명'].value_counts()

이디야     468
스타벅스    257
Name: 브랜드명, dtype: int64

In [11]:
df_cafe_brand = pd.DataFrame(
    df_cafe.groupby(['시군구명', '브랜드명'])['상호명'].count()
).reset_index()
df_cafe_brand.columns = ['구', '브랜드명', '매장수']
df_cafe_brand

,구,브랜드명,매장수
0,가평군,이디야,6
1,고양시 덕양구,스타벅스,11
2,고양시 덕양구,이디야,18
3,고양시 일산동구,스타벅스,14
4,고양시 일산동구,이디야,12
5,고양시 일산서구,스타벅스,11
6,고양시 일산서구,이디야,11
7,과천시,스타벅스,2
8,과천시,이디야,2
9,광명시,스타벅스,7


In [12]:
df_cafe.groupby(['시군구명'])['상호명'].count()

시군구명
가평군          6
고양시 덕양구     29
고양시 일산동구    26
고양시 일산서구    22
과천시          4
광명시         15
광주시         12
구리시         11
군포시         11
김포시         27
남양주시        37
동두천시         5
부천시         49
성남시 분당구     62
성남시 수정구      9
성남시 중원구      4
수원시 권선구     17
수원시 영통구     26
수원시 장안구     11
수원시 팔달구     18
시흥시         25
안산시 단원구     19
안산시 상록구     10
안성시          4
안양시 동안구     21
안양시 만안구     13
양주시         15
양평군          4
여주시          5
연천군          1
오산시          7
용인시 기흥구     28
용인시 수지구     24
용인시 처인구      5
의왕시          6
의정부시        30
이천시         14
파주시         21
평택시         19
포천시         14
하남시         14
화성시         25
Name: 상호명, dtype: int64

In [13]:
df_cafe_gu = pd.DataFrame(df_cafe.groupby(['시군구명'])['상호명'].count())
df_cafe_gu.head()

,상호명
시군구명,
가평군,6
고양시 덕양구,29
고양시 일산동구,26
고양시 일산서구,22
과천시,4


In [14]:
import json
geo_path = 'data/skorea_municipalities_geo_simple.json'
# geo_path = 'data/seoul_municipalities_geo_simple.json'
geo_json = json.load(open(geo_path, encoding='utf-8'))

In [16]:
geo_df = df_cafe

In [17]:
map = folium.Map(location=[geo_df['위도'].mean(), geo_df['경도'].mean()], 
                 zoom_start=12)

map.choropleth(geo_data = geo_json,
               data = df_cafe_gu['상호명'],
               columns = [df_cafe_brand.index, df_cafe_gu['상호명']],
               fill_color = 'Purples',
               key_on = 'feature.properties.name',
               fill_opacity=0.7,
               line_opacity=0.2,
               highlight=True)


for n in geo_df.index:
    popup_name = geo_df.loc[n, '상호명'] + ' - ' + geo_df.loc[n, '도로명주소']
    
    if geo_df.loc[n, '브랜드명'] == '스타벅스' :
        icon_color = 'green'
    else:
        icon_color = 'blue'    
    
    # folium.CircleMarker 혹은 folium.features.CircleMarker 오류가 날 경우
    # --> folium.vector_layer.CircleMarker 를 사용합니다. 
    folium.CircleMarker(
        location=[geo_df.loc[n,'위도'], geo_df.loc[n, '경도']],
        radius=3,
        popup=popup_name,
        color= icon_color,
        fill=True,
        fill_color=icon_color
    ).add_to(map)

map